In [1]:
import yfinance as yf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import datetime
import pandas_ta as ta
from scipy.signal import stft
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None, 'display.max_columns', None)

In [2]:

def fill_missing_dates_daily_2(df, date_col='date'):
    """
    !!! Заменяет значения 'IntradayStd' и 'Volume' нулями в праздники и выходные дни. !!!
    Заполняет пропущенные даты в DataFrame (включая выходные и праздники),
    копируя значения (ffill) с предыдущей даты.
    Удаляет дублирующиеся даты, если они есть, оставляя последнюю запись.
    """
    # Переводим столбец дат в datetime
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')

    # На всякий случай сортируем
    df.sort_values(by=date_col, inplace=True)

    # Удаляем дубликаты по дате, оставляем "последнюю" запись
    df.drop_duplicates(subset=[date_col], keep='last', inplace=True)

    # Ставим дату в индекс
    df.set_index(date_col, inplace=True)

    # Убеждаемся, что индекс уникален
    if not df.index.is_unique:
        raise ValueError("Индекс дат по-прежнему не уникален. Проверь данные.")
    
    df_1 = df.drop(['DayAvgPrice'], axis=1)
    df_2 = df.drop(['IntradayStd', 'Volume'], axis=1)
    df_3 = df.drop(['IntradayStd', 'Volume'], axis=1)

    df_1 = df_1.asfreq(freq ='D',  fill_value = 0.0)
    df_2 = df_2.asfreq(freq ='D',  method='ffill')
    df_3 = df_3.asfreq(freq ='D',  method='bfill')
    
    df_2['IntradayStd'] = df_1['IntradayStd'].copy()
    df_2['Volume'] = df_1['Volume'].copy()
    #df_2['DayAvgPrice_bfill'] = df_3['DayAvgPrice'].copy()

    # Приводим к дневной частоте и заполняем пропуски методом ffill
    #df_daily = df.asfreq('D', method='ffill')

    # Сбрасываем индекс, если нужно вернуть столбец с датами
    df_2.reset_index(inplace=True)
    df_2.rename(columns={'index': date_col}, inplace=True)

    return df_2

In [3]:
def day_of_week_to_imag_2(df):
    """
    day_of_week : pd.Series - день недели в целых числах начиная с пн = О
    period: float — длина одного цикла (например, 7, если недельный цикл)
    """
    #df = df.replace({0: 1.5, 1: 2.2, 2: 2.8, 3: 3., 4: 3.5, 5: 0.5, 6: 0.2})
    #df = df.replace({0: 0.7, 1: 0.8, 2: 1.0, 3: 1.1, 4: 0.9, 5: 0.2, 6: 0.1})
    df = df.replace({0: 3., 1: 4., 2: 4.5, 3: 5., 4: 7., 5: 0.5, 6: 0.1})
    #df = df.replace({0: 10., 1: 14., 2: 17., 3: 25., 4: 27., 5: 0.5, 6: 0.1})

    return df

In [4]:
def spiral_time_indices(date, day_of_week):
    """
    dates : np.array или pd.Series — упорядоченные временные метки (в днях или долях суток)
    period: float — длина одного цикла (например, 7, если недельный цикл)
    """
    # Пусть у нас dates — это массив float (количество дней с какого-то нуля)
    # Если это pd.Datetime, нужно привести к числу дней:
    #t = (date - date[0]).dt.days.astype(float)
    # t = date - date[0]  # если dates уже float
    # t = date
    #real_part = t.astype(float) / 100.

    #real_part = (date.index + 100.) / 100.
    imag_part = day_of_week_to_imag_2(day_of_week)
    #imag_part = day_of_week_to_imag_2(day_of_week) + np.around(np.log(real_part), 3)
    #imag_part = day_of_week_to_imag_2(day_of_week) + real_part / 10.
    
    real_part = date.apply(lambda x: datetime.datetime.timestamp(x) / 86400 / 20000.) # unix days
    #real_part = real_part - real_part[0] + 1.
    #real_part = (date.index + 1.) / 100.
    #real_part = np.log(date.index + 1.)
    #imag_part = day_of_week_to_imag_2(day_of_week)
    #imag_part = 2 * np.pi * day_of_week / 7.
    #imag_part = np.sin(2 * np.pi * real_part/ 7.)
    # real_part = (date.index + 100.) / 100.
    # imag_part = 2 * np.pi * real_part  / 7.
    #real_part = 1.
    #imag_part = 0.
    return real_part, imag_part

In [5]:
# window_cov_eigenvalues(ts, 20) вернёт массив формы (M, n\_features), 
# где M = {len(ts) - window_size + 1}.
# Cov(3f) - ковариационная матрица для 3-х фич, даёт 3 eigenvalues на окно 

def window_cov_eigenvalues(ts, window_size=21):
    """
    Пример: берем окно из time series (одна или несколько фич),
    строим 'ковариацию' признаков за это окно,
    считаем собственные числа (lambda).
    """
    # Будем возвращать список списков (по окну) из eigenvalues
    all_eigs = []

    for start in range(len(ts) - window_size + 1):
        end = start + window_size
        window_data = ts[start:end]  # (window_size,) если ts - 1D

        # Превратим это в 2D вид (если много признаков, shape=(window_size, n_features))
        # Здесь предположим, что ts уже (N, n_features)
        # Но если ts=(N,), сделаем "fake" 2D
        if len(window_data.shape) == 1:
            window_data = window_data.reshape(-1, 1)  # (window_size, 1)

        # Ковариация: shape=(n_features, n_features)
        # rowvar=False => столбцы - это фичи
        cov_mat = np.cov(window_data, rowvar=False)

        # Собственные числа симметричной матрицы => real-valued
        eigvals = np.linalg.eigvalsh(cov_mat)

        all_eigs.append(eigvals)
    
    return np.array(all_eigs)

In [6]:
def create_lambdas_dataset(eigs, horizon=1):
    """
    eigs: (M, d) - M таймшагов, d - кол-во собственных чисел
    return X, y
    X[t] = eigs[t]
    y[t] = eigs[t + horizon]
    """
    X, Y = [], []
    #for i in range(len(eigs) - horizon):
    for i in range(len(eigs)):  # сейчас делаем только Х
        X.append(eigs[i])
        #Y.append(eigs[i + horizon])
    return np.array(X)#, np.array(Y)

In [7]:
# TOEPLITZ
# 1.1. Интерпретация
# 	•	\mathbf{T}[i, i] = r(0) — это просто средняя энергия сигнала на окне (среднее x_n^2).
# 	•	\mathbf{T}[i, j] при i \neq j показывает автокорреляцию на лаг |i-j|.
# 	•	Матрица симметрична, так как |i-j| = |j-i|.
# T(1f) - Toeplitz - матрица для одной фичи, даёт = window_size eigenvalues на окно window_size.

def autocorr_centered(x):
    x = x - np.mean(x)
    c = np.correlate(x, x, mode='full')
    # c[len(x)-1] -- это r(0)
    # Возвращаем часть от lag=0..W-1
    return c[len(x)-1:len(x)-1 + len(x)]

def build_toeplitz_autocorr_centered(window_data):
    """
    Создаёт Toeplitz-матрицу автокорреляции для одномерного окна window_data.
    window_data: np.array формы (W,).
    return: матрица T формы (W, W).
    """
    window_data = window_data - window_data.mean()
    W = len(window_data)
    r = autocorr_centered(window_data)  # shape (W, ), r[0] = r(0), r[1] = r(1), etc.
    # Нормируем?
    r /= (W - np.arange(W))  # иногда нужна такая нормировка

    T = np.zeros((W, W))
    for i in range(W):
        for j in range(W):
            lag = abs(i - j)
            T[i, j] = r[lag]
    return T

In [8]:
# Допустим, у нас есть весь ряд DayAvgPrice длины N. Мы хотим сделать окна длины W и 
# на каждом окне построить Toeplitz-матрицу, затем вычислить её собственные числа.

def toeplitz_eig_dayavg(price_series, window_size=21):
    """
    price_series: np.array формы (N, ), одномерный ряд (DayAvgPrice).
    window_size: длина окна
    return: массив eigenvalues, shape (M, W), где M ~ (N - window_size + 1).
    """
    N = len(price_series)
    all_eigvals = []

    for start in range(N - window_size + 1):
        end = start + window_size
        window_data = price_series[start:end].to_numpy()
        T = build_toeplitz_autocorr_centered(window_data)
        # Собственные числа симметричной (автокорреляционной) матрицы
        eigvals = np.linalg.eigvalsh(T)
        all_eigvals.append(eigvals)

    return np.array(all_eigvals)  # shape (M, window_size)

In [9]:
def lambdas_C_T(df, window_size):

    # вычисляет собственные значения типа С и Т на окне window_size
    # возвращает датафрейм с добавленными столбцами с.з. и без NaN

    #eigs_data = window_cov_eigenvalues(df[['DayAvgPrice', 'IntradayStd', 'Volume']], window_size)
    eigs_data = window_cov_eigenvalues(df[['DayAvgPrice', 'IntradayStd', 'close']], window_size)
    lambdas_C_arr = create_lambdas_dataset(eigs_data, horizon=1)
    lambdas_T_arr = toeplitz_eig_dayavg(df['DayAvgPrice'], window_size)
    df = df.loc[window_size - 1:].reset_index(drop=True)
    
    # Присоединяем eigenvalues
    df = pd.concat([df, pd.DataFrame(lambdas_C_arr, columns=["lambda_C3", "lambda_C2", "lambda_C1"])], axis=1)
    #df_lambda = pd.concat([df_lambda, pd.DataFrame(lambdas_T_arr[:, -3:], columns=["lambda_T3", "lambda_T2", "lambda_T1"])], axis=1)
    df = pd.concat([df, pd.DataFrame(lambdas_T_arr[:, -5:], columns=["lambda_T5", "lambda_T4", "lambda_T3", "lambda_T2", "lambda_T1"])], axis=1)
    
    # Нормировка столбцов lambda_T (как вариант)
    df['lambda_T_sum'] = df['lambda_T1'] + df['lambda_T2'] + df['lambda_T3']
    df['lambda_T1'] = df['lambda_T1'] / df['lambda_T_sum']
    df['lambda_T2'] = df['lambda_T2'] / df['lambda_T_sum']
    df['lambda_T3'] = df['lambda_T3'] / df['lambda_T_sum']
    df = df.drop(['lambda_T_sum', "lambda_T5", "lambda_T4"], axis=1)

    # Нормировка столбцов lambda_C (как вариант)
    df['lambda_C_sum'] = df['lambda_C1'] + df['lambda_C2'] + df['lambda_C3']
    df['lambda_C1'] = df['lambda_C1'] / df['lambda_C_sum']
    df['lambda_C2'] = df['lambda_C2'] / df['lambda_C_sum']
    df['lambda_C3'] = df['lambda_C3'] / df['lambda_C_sum']
    df = df.drop(['lambda_C_sum'], axis=1)



    df = df.dropna()
    df = df.reset_index(drop=True)

    return df
    

In [10]:
# =========================
# Target smoothing
# =========================
from typing import Iterator, Tuple, Optional, List

def smooth_target_ema(
    y: pd.Series, span_fast: int = 5, span_slow: Optional[int] = None
) -> pd.Series:
    """
    EMA-сглаживание таргета.
    - Если указан только span_fast: обычная EMA.
    - Если указан span_slow: двойная EMA (EMA(EMA(y))) — сильнее подавляет шум.
    """
    y1 = y.ewm(span=span_fast, adjust=False).mean()
    if span_slow is None:
        return y1
    y2 = y1.ewm(span=span_slow, adjust=False).mean()
    return y2

In [11]:
# =========================
# Parkinson vola
# =========================
def add_parkinson_features(
    df: pd.DataFrame,
    high_col: str = "high",
    low_col: str  = "low",
    window: int = 20,
    make_derivatives: bool = True,
    winsor_q: float = 0.995,   # срез экстремумов (опционально)
) -> pd.DataFrame:
    df = df.copy()

    # базовая проверка столбцов
    if high_col not in df.columns or low_col not in df.columns:
        raise ValueError(f"Columns `{high_col}` and `{low_col}` must be in df")

    # безопасное отношение high/low
    ratio = (df[high_col] / df[low_col]).replace([np.inf, -np.inf], np.nan)

    # классическая паркинсоновская дисперсия и волатильность
    parkinson_var = (np.log(ratio) ** 2).rolling(window, min_periods=window).mean()
    parkinson_vol = np.sqrt(parkinson_var / (4 * np.log(2)))

    # winsorize для редких всплесков (по желанию)
    # if winsor_q:
    #     q = parkinson_vol.quantile(winsor_q)
    #     parkinson_vol = parkinson_vol.clip(upper=q)

    df["parkinson_vol"] = parkinson_vol.round(6)

    if make_derivatives:
        # сглаживание и динамика (без заглядывания вперёд)
        df["parkinson_vol_ma5"]   = df["parkinson_vol"].rolling(5,  min_periods=5).mean().round(6)
        df["parkinson_vol_ma20"]  = df["parkinson_vol"].rolling(20, min_periods=20).mean().round(6)
        df["parkinson_vol_diff1"] = df["parkinson_vol"].diff(1).round(6)
        df["parkinson_vol_lag1"]  = df["parkinson_vol"].shift(1).round(6)

    # финальные NaN на ранних барах — оставляем; их отфильтрует твой датасет окон
    return df

In [12]:
# Wavelet/STFT энергия (устойчивые спектральные фичи)(эскиз на STFT; для продакшена лучше кэшировать)

def stft_energy_tail(x: np.ndarray, w: int = 64) -> list:
    """
    Возвращает 3 числа — энергию сигнала в низких, средних и высоких частотах
    за последние w наблюдений x.
    """
    if len(x) < w or np.isnan(x).any():
        return [np.nan, np.nan, np.nan]
    
    # Берём последний кусок длиной w
    x_tail = x[-w:]
    f, t, Z = stft(x_tail, nperseg=w//2)
    S = np.abs(Z) ** 2  # спектральная плотность мощности

    # усредняем по времени
    S_mean = np.mean(S, axis=1)

    # делим на 3 диапазона частот
    bands = np.array_split(S_mean, 3)
    energies = [np.mean(b) for b in bands]
    return energies

In [13]:
# bild_advanced_features

# ========= helpers =========
def _safe_div(a, b):
    return a / b.replace(0, np.nan)

def _rolling_cov(x, y, w):
    return (x.rolling(w).mean()*y.rolling(w).mean() - (x*y).rolling(w).mean()) * (-1)  # не используем, см. ниже

# ========= базовые конструкции =========
def realized_vol(ret, w):
    return ret.rolling(w, min_periods=w).std(ddof=1)

def bbands(close, w=20, k=2.0):
    ma = close.rolling(w, min_periods=w).mean()
    sd = close.rolling(w, min_periods=w).std(ddof=1)
    bb_up = ma + k*sd
    bb_mid = ma
    bb_low = ma - k*sd
    # безразмерные и устойчивые к масштабу:
    pct_b = (close - bb_low) / (bb_up - bb_low)
    bandwidth = _safe_div(bb_up - bb_low, bb_mid.abs())
    return bb_low, bb_mid, bb_up, pct_b, bandwidth

def atr(df, n=14):
    h, l, c = df['high'], df['low'], df['close']
    prev_c = c.shift(1)
    tr = pd.concat([
        (h - l),
        (h - prev_c).abs(),
        (l - prev_c).abs()
    ], axis=1).max(axis=1)
    return tr.rolling(n, min_periods=n).mean()

def adx(df, n=14):
    h, l, c = df['high'], df['low'], df['close']
    up_move = h.diff()
    down_move = -l.diff()
    plus_dm = np.where((up_move > down_move) & (up_move > 0), up_move, 0.0)
    minus_dm = np.where((down_move > up_move) & (down_move > 0), down_move, 0.0)
    tr = atr(df, n)*n  # вернуть к сумме TR
    plus_di = 100 * pd.Series(plus_dm, index=h.index).rolling(n, min_periods=n).sum() / tr
    minus_di= 100 * pd.Series(minus_dm, index=h.index).rolling(n, min_periods=n).sum() / tr
    dx = 100 * (plus_di - minus_di).abs() / (plus_di + minus_di).replace(0, np.nan)
    adx = dx.rolling(n, min_periods=n).mean()
    return adx, plus_di, minus_di

def choppiness(df, n=14):
    _atr = atr(df, n)
    hi = df['high'].rolling(n, min_periods=n).max()
    lo = df['low'].rolling(n, min_periods=n).min()
    denom = (hi - lo).replace(0, np.nan)
    return 100 * np.log(_atr.rolling(n, min_periods=n).sum() / denom) / np.log(n)

def garman_klass_sigma(df, w=20):
    u = np.log(df['high']/df['open'])
    d = np.log(df['low']/df['open'])
    c = np.log(df['close']/df['open'])
    var = (0.5*(u - d)**2 - (2*np.log(2)-1)*c**2).rolling(w, min_periods=w).mean()
    return np.sqrt(var.clip(lower=0))

def rogers_satchell_sigma(df, w=20):
    u = np.log(df['high']/df['close'].shift(1))
    d = np.log(df['low']/df['close'].shift(1))
    c = np.log(df['close']/df['open'])
    var = (u*(u-c) + d*(d-c)).rolling(w, min_periods=w).mean()
    return np.sqrt(var.clip(lower=0))

def yang_zhang_sigma(df, w=20):
    oc = np.log(df['open']/df['close'].shift(1))
    co = np.log(df['close']/df['open'])
    k = 0.34/(1.34 + (w+1)/(w-1))
    oc2 = oc.rolling(w, min_periods=w).var(ddof=1)
    co2 = co.rolling(w, min_periods=w).var(ddof=1)
    rs  = (np.log(df['high']/df['close']) * np.log(df['high']/df['open'])
          + np.log(df['low']/df['close'])  * np.log(df['low']/df['open'])).rolling(w, min_periods=w).mean()
    var = oc2 + k*co2 + (1-k)*rs
    return np.sqrt(var.clip(lower=0))

def amihud_illiquidity(ret, volume, w=20):
    illiq = _safe_div(ret.abs(), volume.replace(0, np.nan))
    return illiq.rolling(w, min_periods=w).mean()

def roll_measure(close, w=20):
    dp = close.diff()
    cov = (dp * dp.shift(1)).rolling(w, min_periods=w).mean()
    sigma_spread = -2 * cov
    sigma_spread = sigma_spread.mask(sigma_spread < 0)  # по определению
    return sigma_spread

def vol_of_vol(vol_series, w=20):
    return vol_series.rolling(w, min_periods=w).std(ddof=1)

def kalman_slope(y, q=1e-5, r=1e-2):
    """
    Простая 1D модель: уровень+наклон. Возвращает оценку slope.
    q,r — дисперсии процесса/измерения.
    """
    n = len(y)
    if n == 0: return pd.Series(dtype=float)
    # состояние [level, slope]
    x = np.array([y.fillna(method='ffill').iloc[0], 0.0], dtype=float)
    P = np.eye(2)
    F = np.array([[1.0, 1.0],
                  [0.0, 1.0]])
    Q = q * np.array([[0.25, 0.5],
                      [0.5,  1.0]])
    H = np.array([[1.0, 0.0]])
    R = np.array([[r]])
    slopes = []
    for z in y.fillna(method='ffill').values:
        # predict
        x = F @ x
        P = F @ P @ F.T + Q
        # update
        yk = z - (H @ x)
        S = H @ P @ H.T + R
        K = (P @ H.T) @ np.linalg.inv(S)
        x = x + (K @ yk).ravel()
        P = (np.eye(2) - K @ H) @ P
        slopes.append(x[1])
    return pd.Series(slopes, index=y.index)

# ========= главный конструктор фич =========
def build_advanced_features(df: pd.DataFrame,
                            price_col: str = 'close',
                            w_fast: int = 14,
                            w_slow: int = 20) -> pd.DataFrame:
    """
    Ожидает df с колонками: open, high, low, close, volume (индекс — datetime).
    Возвращает df с новыми фичами (без нормировки).
    """
    df = df.copy()
    for col in ['open','high','low','close','Volume']:
        if col not in df.columns:
            raise ValueError(f"Missing required column: {col}")
    df = df.sort_index()

    # 0) базовые ретёрны
    df['ret'] = np.log(df['close']/df['close'].shift(1))
    df['ret_overnight'] = np.log(df['open']/df['close'].shift(1))
    df['ret_intraday']  = np.log(df['close']/df['open'])

    # 1) волатильности (улучшенные)
    df['gk_sigma']  = garman_klass_sigma(df, w=w_slow)
    df['rs_sigma']  = rogers_satchell_sigma(df, w=w_slow)
    df['yz_sigma']  = yang_zhang_sigma(df, w=w_slow)

    # 2) режим/тренд
    df['adx'], df['di_plus'], df['di_minus'] = adx(df, n=w_fast)
    df['chop'] = choppiness(df, n=w_fast)
    df['kalm_slope'] = kalman_slope(df[price_col])

    # 3) объёмы/ликвидность
    # df['amihud'] = amihud_illiquidity(df['ret'], df['Volume'], w=w_slow)
    # df['roll_sigma_spread'] = roll_measure(df['close'], w=w_slow)

    # 4) вола-волы и связки
    df['rv20'] = realized_vol(df['ret'], w=w_slow)
    df['vol_of_vol'] = vol_of_vol(df['rv20'], w=w_fast)

    # 5) Bollinger в безразмерном виде
    bb_low, bb_mid, bb_up, pct_b, bw = bbands(df['close'], w=w_slow, k=2.0)
    df['bb_pct_b'] = pct_b
    df['bb_bandwidth'] = bw

    # 6) взаимодействие цены и объёма
    dlog_vol = np.log1p(df['Volume']).diff()
    df['corr_ret_dlogvol'] = df['ret'].rolling(w_slow, min_periods=w_slow).corr(dlog_vol)

    # аккуратный старт: удалим строки до прогрева окон (потом они удаляются)
    # warmup = max(w_fast, w_slow) + 1
    # return df.iloc[warmup:].copy()
    return df.copy()

### Дополнение архивного файла свежими данными.

In [319]:
# Set variables
ticker = "BAYN.DE"
start_date = "2020-09-04"
end_date = "2020-09-27"

In [ ]:
# Download new stock data
data = yf.download(ticker, start=start_date, end=end_date, interval='1h', auto_adjust=True)
data.columns = [f'{Price}' for Price, Ticker in data.columns]
data['date'] = data.index.date
data = data.reset_index().drop(columns=['Datetime'])
data.columns = data.columns.str.lower()
data

In [ ]:
# load old data from data_archiv
data_old = pd.read_csv('../data_archiv/MBG/MBG_to12_06_2025.csv')
data_old.tail(10)

In [ ]:
# load old data from data_archiv
data_new = pd.read_csv('../data_archiv/MBG/MB_13_06_2025to12_08_2025.csv')
data_new.head(10)

In [ ]:
# Объединяем с новыми данными
data_fresh = pd.concat([data_old, data_new], ignore_index=True)
data_fresh.tail(20)

In [59]:
# Save
data_fresh.to_csv('../data_archiv/MBG/MBG_fresh.csv', index=False)

##### Загружаем свежие данные с Yahoo Finance

In [136]:
# Set variables
ticker = "DTG.DE"
start_date = "2025-11-14"
end_date = "2025-11-15"

In [137]:
# Download new stock data
data = yf.download(ticker, start=start_date, end=end_date, interval='1h', auto_adjust=True)
data.columns = [f'{Price}' for Price, Ticker in data.columns]
data['date'] = data.index.date
data = data.reset_index().drop(columns=['Datetime'])
data.columns = data.columns.str.lower()
data

[*********************100%***********************]  1 of 1 completed


,close,high,low,open,volume,date
0,35.490002,35.639999,35.369999,35.450001,0,2025-11-14
1,35.400002,35.500000,35.360001,35.500000,25805,2025-11-14
2,35.349998,35.470001,35.330002,35.410000,38157,2025-11-14
3,34.950001,35.340000,34.869999,35.340000,91519,2025-11-14
4,34.860001,35.009998,34.830002,34.939999,63203,2025-11-14
5,35.049999,35.160000,34.799999,34.849998,75638,2025-11-14
6,35.139999,35.139999,34.980000,35.060001,39205,2025-11-14
7,35.310001,35.369999,35.070000,35.130001,48565,2025-11-14
8,35.340000,35.369999,35.259998,35.310001,24330,2025-11-14


In [138]:
# load old data from data_archiv
data_old = pd.read_csv('../data_archiv/DTG/DTG_fresh.csv')
data_old.tail(10)

,close,high,low,open,volume,date
8698,36.029999,36.139999,36.029999,36.099998,37288,2025-11-12
8699,36.250000,36.369999,36.150002,36.189999,0,2025-11-13
8700,36.150002,36.310001,36.040001,36.250000,138735,2025-11-13
8701,35.980000,36.130001,35.970001,36.130001,19781,2025-11-13
8702,36.020000,36.020000,35.910000,35.980000,31517,2025-11-13
8703,35.959999,36.049999,35.939999,35.990002,10612,2025-11-13
8704,35.910000,36.049999,35.869999,35.939999,20074,2025-11-13
8705,35.959999,36.000000,35.860001,35.900002,32631,2025-11-13
8706,35.779999,35.950001,35.740002,35.950001,42725,2025-11-13
8707,35.810001,35.919998,35.730000,35.770000,70158,2025-11-13


In [139]:
data_old.isna().value_counts()

close  high   low    open   volume  date 
False  False  False  False  False   False    8708
dtype: int64

In [140]:
# Объединяем с новыми данными
data_fresh = pd.concat([data_old, data], ignore_index=True)
data_fresh.tail(20)

,close,high,low,open,volume,date
8697,36.090000,36.380001,35.970001,36.349998,58665,2025-11-12
8698,36.029999,36.139999,36.029999,36.099998,37288,2025-11-12
8699,36.250000,36.369999,36.150002,36.189999,0,2025-11-13
8700,36.150002,36.310001,36.040001,36.250000,138735,2025-11-13
8701,35.980000,36.130001,35.970001,36.130001,19781,2025-11-13
8702,36.020000,36.020000,35.910000,35.980000,31517,2025-11-13
8703,35.959999,36.049999,35.939999,35.990002,10612,2025-11-13
8704,35.910000,36.049999,35.869999,35.939999,20074,2025-11-13
8705,35.959999,36.000000,35.860001,35.900002,32631,2025-11-13
8706,35.779999,35.950001,35.740002,35.950001,42725,2025-11-13


In [141]:
# Save
data_fresh.to_csv('../data_archiv/DTG/DTG_fresh.csv', index=False)

#### START DATA PREP

In [184]:
data_mb = pd.read_csv('../data_archiv/DTG/DTG_fresh.csv')

In [185]:
data_mb[['open','high','low','close']] = data_mb[['open','high','low','close']].apply(pd.to_numeric, errors='coerce')
# И datetime — это pandas.Timestamp с часовыми барами
df = data_mb.copy()
df['datetime'] = pd.to_datetime(df['date'])
df = df.set_index('datetime')

# === Ресемплинг на дневную частоту ===
df_daily = pd.DataFrame()

df_daily['open']  = df['open'].resample('1D').first()
df_daily['high']  = df['high'].resample('1D').max()
df_daily['low']   = df['low'].resample('1D').min()
df_daily['close'] = df['close'].resample('1D').last()
df_daily['Volume']= df['volume'].resample('1D').sum()

df_daily['DayAvgPrice']= df[['open', 'high', 'low', 'close']].resample('1D').mean().mean(axis=1)

df_daily['IntradayStd'] = df.resample('1D').apply(
    lambda day: day[['open','high','low','close']].to_numpy(dtype=float).flatten().std(ddof=1)
).rename('IntradayStd').to_frame()

# df_daily['IntradayStd'] = df.resample('1D').apply(
#     lambda x: np.std(x[['open','high','low','close']].values.flatten(), ddof=1)
# ).to_frame(name='IntradayStd')

# === Убираем пустые дни (например, выходные) ===
df_daily = df_daily.dropna(subset=['open', 'high', 'low', 'close'])

# === Сбрасываем индекс, если нужно для модели ===
df_daily = df_daily.reset_index().rename(columns={'datetime': 'date'})

df_daily.tail()

,date,open,high,low,close,Volume,DayAvgPrice,IntradayStd
966,2025-11-10,35.169998,36.130001,34.980000,35.480000,827914,35.753889,0.282107
967,2025-11-11,35.560001,36.279999,35.529999,35.980000,496435,35.886111,0.159271
968,2025-11-12,36.240002,36.660000,35.779999,36.029999,459630,36.299444,0.184235
969,2025-11-13,36.189999,36.369999,35.730000,35.810001,366233,35.998056,0.155321
970,2025-11-14,35.450001,35.639999,34.799999,35.340000,406422,35.215278,0.229179


In [ ]:
# df_daily = fill_missing_dates_daily_2(df_daily, date_col='date') # 'DayAvgPrice' и 'IntradayStd'=0 в неторговые дни
# df_daily.tail(10)

In [ ]:
# !!! Только в случае попадания выходных или праздников в конец датасета дополняем датасет вручную !!!

# last_index = df_daily.index.max()
# df_daily.loc[last_index + 1] = [df_daily['date'][last_index] + pd.DateOffset(days=1), df_daily['open'][last_index], df_daily['high'][last_index], df_daily['low'][last_index], df_daily['close'][last_index], df_daily['DayAvgPrice'][last_index], 0., 0,]
# df_daily.loc[last_index + 2] = [df_daily['date'][last_index] + pd.DateOffset(days=2), df_daily['open'][last_index], df_daily['high'][last_index], df_daily['low'][last_index], df_daily['close'][last_index], df_daily['DayAvgPrice'][last_index], 0., 0,]
# df_daily.tail()

### Генерация признаков

In [186]:
# Размер окна для поиска паттернов
window_size = 128

##### Старые признаки

In [187]:
# Стандартные
df_daily['day_of_week'] = df_daily['date'].apply(lambda x: x.day_of_week)
df_daily['day_of_year'] = df_daily['date'].apply(lambda x: x.day_of_year)
df_daily['Log_Profit'] = np.log(df_daily['DayAvgPrice'].shift(1) / df_daily['DayAvgPrice'])
df_daily['DayAvgPrice_diff'] = df_daily['DayAvgPrice'] - df_daily['DayAvgPrice'].shift(1)
df_daily['DayAvgPrice_2diff'] = df_daily['DayAvgPrice_diff'] - df_daily['DayAvgPrice_diff'].shift(1)

In [189]:
# Полиномиальные
df_daily['POLY_1'] = np.sqrt(df_daily['DayAvgPrice']) + df_daily['IntradayStd'] * 100.
df_daily['POLY_2'] = df_daily['DayAvgPrice'] + (df_daily['IntradayStd'] * 10.) ** 2
df_daily['POLY_3'] = df_daily['DayAvgPrice'] ** 2 / 1000. + df_daily['Volume'] / 100000.

In [190]:
# === Комплексные
df_daily['real_time'], df_daily['imag_time'] = spiral_time_indices(df_daily['date'], df_daily['day_of_week'])

# === Комплексные циклы ===
t = np.arange(len(df_daily))
df_daily['c_week_real'] = np.cos(2 * np.pi * t / 7)
df_daily['c_week_imag'] = np.sin(2 * np.pi * t / 7)

df_daily['c_month_real'] = np.cos(2 * np.pi * t / 30.44)
df_daily['c_month_imag'] = np.sin(2 * np.pi * t / 30.44)

df_daily['c_quarter_real'] = np.cos(2 * np.pi * t / 91.31)
df_daily['c_quarter_imag'] = np.sin(2 * np.pi * t / 91.31)

df_daily['c_year_real'] = np.cos(2 * np.pi * t / 365.25)
df_daily['c_year_imag'] = np.sin(2 * np.pi * t / 365.25)

##### Новые признаки

In [191]:
# добавляем фичи на волатильность Паркинсона
df_daily = add_parkinson_features(df_daily, high_col='high', low_col='low', window=20)

In [192]:
# Сглаживание DayAvgPrice: rolling (5, 10, 20) и EMA
df_daily['DayAvgPrice_roll5']  = df_daily['DayAvgPrice'].rolling(5,  min_periods=5).mean()
df_daily['DayAvgPrice_roll10'] = df_daily['DayAvgPrice'].rolling(10, min_periods=10).mean()
df_daily['DayAvgPrice_roll20'] = df_daily['DayAvgPrice'].rolling(20, min_periods=20).mean()

df_daily['DayAvgPrice_ema5']   = df_daily['DayAvgPrice'].ewm(span=5,  adjust=False, min_periods=5).mean()
df_daily['DayAvgPrice_ema20']  = df_daily['DayAvgPrice'].ewm(span=20, adjust=False, min_periods=20).mean()

In [193]:
# Сглаживание IntradayStd: rolling (5, 10, 20) и EMA
df_daily['IntradayStd_roll5']  = df_daily['IntradayStd'].rolling(5,  min_periods=5).mean()
df_daily['IntradayStd_roll10'] = df_daily['IntradayStd'].rolling(10, min_periods=10).mean()
df_daily['IntradayStd_roll20'] = df_daily['IntradayStd'].rolling(20, min_periods=20).mean()

df_daily['IntradayStd_ema5']   = df_daily['IntradayStd'].ewm(span=5,  adjust=False, min_periods=5).mean()
df_daily['IntradayStd_ema20']  = df_daily['IntradayStd'].ewm(span=20, adjust=False, min_periods=20).mean()

In [194]:
# Новые признаки

w = window_size

# Лаги/доходности
df_daily['log_ret_1'] = np.log(df_daily['DayAvgPrice']).diff()
for k in [1,2,3,4,5,6,7,10]:
    df_daily[f'DAP_{k}'] = df_daily['DayAvgPrice'].shift(k)
    df_daily[f'log_ret_{k}']  = df_daily['log_ret_1'].rolling(k).sum()  # k-step cum return

# Оконные агрегаты
df_daily['mean_w'] = df_daily['DayAvgPrice'].rolling(w, min_periods=w).mean()
df_daily['std_w']  = df_daily['DayAvgPrice'].rolling(w, min_periods=w).std()
df_daily['z_w']    = (df_daily['DayAvgPrice'] - df_daily['mean_w']) / (df_daily['std_w'] + 1e-9)
df_daily['q10_w']  = df_daily['DayAvgPrice'].rolling(w, min_periods=w).quantile(0.10)
df_daily['q90_w']  = df_daily['DayAvgPrice'].rolling(w, min_periods=w).quantile(0.90)

# Наклон тренда (скользящая линрегрессия через cov/var)
def rolling_slope(s, w):
    x = np.arange(len(s))
    mx = pd.Series(x).rolling(w, min_periods=w).mean().values
    my = s.rolling(w, min_periods=w).mean().values
    cov = (pd.Series(x)*s).rolling(w, min_periods=w).mean().values - mx*my
    var = pd.Series(x).rolling(w, min_periods=w).var().values + 1e-9
    return cov/var
df_daily['slope_w'] = rolling_slope(df_daily['DayAvgPrice'], w)

# Волатильность
# df_daily['parkinson_vol'] = (np.log(df_daily['high']/df_daily['low'])**2).rolling(w, min_periods=w).mean()
df_daily['vol_w'] = df_daily['log_ret_1'].rolling(w, min_periods=w).std()

# Объём
df_daily['vol_z'] = (df_daily['Volume'] - df_daily['Volume'].rolling(w).mean()) / (df_daily['Volume'].rolling(w).std() + 1e-9)

# Календарь (синусы – без утечек)
d = pd.to_datetime(df_daily['date'])
df_daily['dow_sin'] = np.sin(2*np.pi*d.dt.dayofweek/7)
df_daily['dow_cos'] = np.cos(2*np.pi*d.dt.dayofweek/7)
df_daily['moy_sin'] = np.sin(2*np.pi*(d.dt.month-1)/12)
df_daily['moy_cos'] = np.cos(2*np.pi*(d.dt.month-1)/12)

In [195]:
# Техиндикаторы с pandas_ta:
df_daily['rsi14'] = ta.rsi(df_daily['DayAvgPrice'], length=14)
macd = ta.macd(df_daily['DayAvgPrice'], fast=12, slow=26, signal=9)
df_daily[['macd','macd_signal','macd_hist']] = macd[['MACD_12_26_9','MACDs_12_26_9','MACDh_12_26_9']]
bb = ta.bbands(df_daily['DayAvgPrice'], length=20)
df_daily[['bb_low','bb_mid','bb_up']] = bb[['BBL_20_2.0','BBM_20_2.0','BBU_20_2.0']]
df_daily['atr14'] = ta.atr(df_daily['high'], df_daily['low'], df_daily['DayAvgPrice'], length=14)

In [196]:
# Hilbert-фаза/амплитуда (сигнал в комплексной форме)
from scipy.signal import hilbert
analytic = pd.Series(df_daily['DayAvgPrice']).rolling(w, min_periods=w).apply(
    lambda x: np.nan if x.isna().any() else np.angle(hilbert(x.values))[-1], raw=False
)
df_daily['phi_hilbert'] = analytic  # фаза конца окна
# при желании — дельта фазы:
df_daily['dphi_hilbert'] = df_daily['phi_hilbert'].diff()

In [197]:
# Wavelet/STFT энергия (устойчивые спектральные фичи)(эскиз на STFT; для продакшена лучше кэшировать)

prices = df_daily['DayAvgPrice'].values
window_stft = 128  # длина окна для rolling STFT (≈ полгода дневных данных, например)

energy_low = []
energy_mid = []
energy_high = []

for i in range(len(prices)):
    x = prices[:i+1]  # берём префикс до текущего момента
    e = stft_energy_tail(x, w=window_stft)
    energy_low.append(e[0])
    energy_mid.append(e[1])
    energy_high.append(e[2])

df_daily['stft_energy_low']  = energy_low
df_daily['stft_energy_mid']  = energy_mid
df_daily['stft_energy_high'] = energy_high

# Вариант для коротких df (len(df) >= 128)
df_daily[['stft_energy_low', 'stft_energy_mid', 'stft_energy_high']] = (
    df_daily[['stft_energy_low', 'stft_energy_mid', 'stft_energy_high']].fillna(0)
)

# !!!Вариант для длинных df (len(df) >> 128) - лучше сделать dropna после создания всех фич!!!
#df_daily = df_daily.dropna(subset=['stft_energy_low', 'stft_energy_mid', 'stft_energy_high'])

In [198]:
len(df_daily)

971

In [199]:
# advanced_features

# df — ваш OHLCV с datetime индексом (часовой/дневной — не важно)
df_daily = build_advanced_features(df_daily, price_col='DayAvgPrice', w_fast=14, w_slow=20)

# Дальше: добавляйте нужные колонки в real_cols и нормируйте по train-окну
new_cols = [
    'gk_sigma','rs_sigma','yz_sigma','adx','chop','kalm_slope',
    'rv20','vol_of_vol','bb_pct_b','bb_bandwidth',
    'ret_overnight','ret_intraday','corr_ret_dlogvol'
]

In [200]:
# Спектральные признаки(собственные значения TOEPLITZ и CORR матриц) + на весь df.dropna()!!!
df_daily = lambdas_C_T(df_daily, window_size)

In [ ]:
# df_daily.isna().sum()

##### Создание целевой переменной Target

In [201]:
# Создаём Target, пустые последние значения заменяются предыдущим непустым ffill
time_shift = 1 # другие сдвиги реализуем в рабочем блоке
df_daily['Target'] = df_daily['DayAvgPrice'].shift( - time_shift).fillna(method='ffill')   # fffill
#df_daily['DayAvgPrice'] = df_daily['DayAvgPrice_bfill']                                   # b?fill
#df_daily['Target'] = df_daily['DayAvgPrice_bfill'].shift( - time_shift).fillna(0)
#df_daily = df_daily.drop(['DayAvgPrice_bfill'], axis=1)                                   # ?bfill
#df_daily[['date', 'DayAvgPrice', 'Target', 'DayAvgPrice_bfill']].tail(10)
df_daily[['date', 'DayAvgPrice', 'Target']].tail(10)

,date,DayAvgPrice,Target
833,2025-11-03,34.606944,33.886111
834,2025-11-04,33.886111,34.891389
835,2025-11-05,34.891389,35.163611
836,2025-11-06,35.163611,34.328333
837,2025-11-07,34.328333,35.753889
838,2025-11-10,35.753889,35.886111
839,2025-11-11,35.886111,36.299444
840,2025-11-12,36.299444,35.998056
841,2025-11-13,35.998056,35.215278
842,2025-11-14,35.215278,35.215278


In [202]:
# Сглаженный таргет (не меняем исходный Target, а создаём Target_smooth)
ema_fast: int = 5
ema_slow: Optional[int] = None
df_daily["Target_smooth"] = smooth_target_ema(df_daily['Target'], span_fast=ema_fast, span_slow=ema_slow)

In [203]:
# Добавляем количество дней между наблюдениями
df_daily['days_since_prev'] = df_daily['date'].diff().dt.days.fillna(1)

In [204]:
len(list(df_daily.columns))

107

In [90]:
df_daily.describe()

,open,high,low,close,Volume,DayAvgPrice,IntradayStd,day_of_week,day_of_year,Log_Profit,DayAvgPrice_diff,DayAvgPrice_2diff,POLY_1,POLY_2,POLY_3,real_time,imag_time,c_week_real,c_week_imag,c_month_real,c_month_imag,c_quarter_real,c_quarter_imag,c_year_real,c_year_imag,parkinson_vol,parkinson_vol_ma5,parkinson_vol_ma20,parkinson_vol_diff1,parkinson_vol_lag1,DayAvgPrice_roll5,DayAvgPrice_roll10,DayAvgPrice_roll20,DayAvgPrice_ema5,DayAvgPrice_ema20,IntradayStd_roll5,IntradayStd_roll10,IntradayStd_roll20,IntradayStd_ema5,IntradayStd_ema20,log_ret_1,DAP_1,DAP_2,log_ret_2,DAP_3,log_ret_3,DAP_4,log_ret_4,DAP_5,log_ret_5,DAP_6,log_ret_6,DAP_7,log_ret_7,DAP_10,log_ret_10,mean_w,std_w,z_w,q10_w,q90_w,slope_w,vol_w,vol_z,dow_sin,dow_cos,moy_sin,moy_cos,rsi14,macd,macd_signal,macd_hist,bb_low,bb_mid,bb_up,atr14,phi_hilbert,dphi_hilbert,stft_energy_low,stft_energy_mid,stft_energy_high,ret,ret_overnight,ret_intraday,gk_sigma,rs_sigma,yz_sigma,adx,di_plus,di_minus,chop,kalm_slope,rv20,vol_of_vol,bb_pct_b,bb_bandwidth,corr_ret_dlogvol,lambda_C3,lambda_C2,lambda_C1,lambda_T3,lambda_T2,lambda_T1,Target,Target_smooth,days_since_prev
count,932.000000,932.000000,932.000000,932.000000,9.320000e+02,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,9.320000e+02,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,9.320000e+02,9.320000e+02,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000
mean,33.662296,34.074045,33.249753,33.665730,1.626297e+06,33.667936,0.199217,2.010730,185.790773,-0.000424,0.012611,-0.000057,25.705225,39.020258,17.426012,0.986939,4.707082,-0.000967,4.655405e-04,-0.009396,-0.002459,-0.018809,-0.001680,-0.088947,0.085110,0.016371,0.016411,0.016542,-0.000020,0.016391,33.642978,33.613724,33.555042,33.643414,33.568139,0.199251,0.199268,0.200310,0.199248,0.200130,0.000424,33.655326,33.642658,0.000846,33.630475,0.001254,33.618496,0.001656,33.607812,0.002024,33.595238,0.002459,33.584138,0.002836,33.550379,0.004006,33.550258,1.130109,0.044703,32.171729,34.925562,0.010797,0.017137,-0.021025,0.354926,-0.087022,-1.419748e-02,-8.973326e-02,51.678324,0.054979,0.046804,0.008175,31.414595,33.555042,35.695490,0.944181,0.002746,0.000312,90.115342,0.231197,0.113292,0.000422,0.000334,0.000087,0.016431,0.022729,0.020004,35.436490,24.694092,23.598960,47.702435,0.011684,0.018620,0.002164,0.511871,0.132962,-0.068028,0.003314,0.017377,0.979309,0.122445,0.339217,0.538338,33.680459,33.658637,1.429185
std,5.433288,5.479246,5.374738,5.437765,1.126751e+06,5.435745,0.117688,1.408846,99.323773,0.017977,0.619459,0.841541,11.872482,11.004661,11.232760,0.019283,1.321569,0.707722,7.072496e-01,0.706336,0.708569,0.712454,0.702229,0.701554,0.702655,0.005403,0.005426,0.005347,0.000714,0.005438,5.421464,5.410312,5.385909,5.406436,5.286285,0.077548,0.068785,0.063219,0.076724,0.061975,0.017977,5.443960,5.451532,0.026459,5.459195,0.032476,5.466775,0.037566,5.475109,0.042002,5.484981,0.046026,5.492817,0.049545,5.519772,0.059922,5.381132,0.569103,1.357297,5.225023,5.646875,0.155847,0.006122,1.000671,0.514230,0.776501,7.169590e-01,6.919403e-01,14.037166,0.783897,0.751190,

In [134]:
df_daily.head()

,date,open,high,low,close,Volume,DayAvgPrice,IntradayStd,day_of_week,day_of_year,Log_Profit,DayAvgPrice_diff,DayAvgPrice_2diff,POLY_1,POLY_2,POLY_3,real_time,imag_time,c_week_real,c_week_imag,c_month_real,c_month_imag,c_quarter_real,c_quarter_imag,c_year_real,c_year_imag,parkinson_vol,parkinson_vol_ma5,parkinson_vol_ma20,parkinson_vol_diff1,parkinson_vol_lag1,DayAvgPrice_roll5,DayAvgPrice_roll10,DayAvgPrice_roll20,DayAvgPrice_ema5,DayAvgPrice_ema20,IntradayStd_roll5,IntradayStd_roll10,IntradayStd_roll20,IntradayStd_ema5,IntradayStd_ema20,log_ret_1,DAP_1,DAP_2,log_ret_2,DAP_3,log_ret_3,DAP_4,log_ret_4,DAP_5,log_ret_5,DAP_6,log_ret_6,DAP_7,log_ret_7,DAP_10,log_ret_10,mean_w,std_w,z_w,q10_w,q90_w,slope_w,vol_w,vol_z,dow_sin,dow_cos,moy_sin,moy_cos,rsi14,macd,macd_signal,macd_hist,bb_low,bb_mid,bb_up,atr14,phi_hilbert,dphi_hilbert,stft_energy_low,stft_energy_mid,stft_energy_high,ret,ret_overnight,ret_intraday,gk_sigma,rs_sigma,yz_sigma,adx,di_plus,di_minus,chop,kalm_slope,rv20,vol_of_vol,bb_pct_b,bb_bandwidth,corr_ret_dlogvol,lambda_C3,lambda_C2,lambda_C1,lambda_T3,lambda_T2,lambda_T1,Target,Target_smooth,days_since_prev
0,2022-07-29,26.600,26.985,26.570,26.59,1027153,26.737639,0.117190,4,210,-0.022770,0.601944,-0.028472,16.889869,28.110994,10.986431,0.960096,7.0,-0.222521,0.974928,0.279031,0.960282,-0.815678,0.578506,-0.590044,0.807371,0.025673,0.025815,0.024389,-0.000487,0.026160,25.828056,25.577028,24.831062,25.993706,25.445195,0.181709,0.203698,0.231644,0.178947,0.208529,0.022770,26.135694,25.505278,0.047187,25.075833,0.064168,25.685833,0.040133,25.551389,0.045381,25.462222,0.048876,25.683333,0.040230,24.219583,0.098911,26.787799,2.826791,-0.017744,23.728528,31.702486,-0.026379,0.027282,-0.621276,-0.433884,-0.900969,1.224647e-16,-1.000000,59.645167,-0.005002,-0.321081,0.316079,22.984941,24.831062,26.677184,0.982564,-0.286948,0.027044,61.761396,0.202887,0.089728,0.007550,0.007926,-0.000376,0.023240,0.026916,0.025983,26.972219,29.047430,11.093688,53.453466,0.142538,0.033393,0.000725,0.899519,0.173053,-0.105973,0.000645,0.002835,0.996520,0.129691,0.380312,0.489997,26.904028,26.904028,1.0
1,2022-08-01,26.700,27.185,26.615,27.02,1479570,26.904028,0.145002,0,213,-0.006204,0.166389,-0.435556,19.687063,29.006572,15.519527,0.960246,3.0,-0.900969,0.433884,0.076298,0.997085,-0.853524,0.521053,-0.603845,0.797102,0.025685,0.025810,0.024784,0.000012,0.025673,26.071694,25.787069,24.924958,26.297147,25.584131,0.173077,0.202573,0.232555,0.167632,0.202478,0.006204,26.737639,26.135694,0.028974,25.505278,0.053391,25.075833,0.070371,25.685833,0.046336,25.551389,0.051584,25.462222,0.055080,24.803611,0.081287,26.750903,2.793764,0.054809,23.728528,31.702486,-0.024565,0.027290,-0.286681,0.000000,1.000000,-5.000000e-01,-0.866025,60.700830,0.105694,-0.235726,0.341420,22.869545,24.924958,26.980372,0.953095,-0.290760,-0.003812,61.620540,0.207358,0.099177,0.016042,0.004128,0.011914,0.023230,0.026933,0.025913,30.265104,31.825588,6.845965,50.899823,0.167565,0.033491,0.000732,0.934271,0.188799,-0.121758,0.000663,0.002886,0.996451,0.132952,0.381615,0.485433,27.198472,27.002176,3.0
2,2022-08-02,26.850,27.530,26.705,27.32,996791,27.198472,0.206092,1,214,-0.010885,0.294445,0.128056,25.824441,31.445874,10.707667,0.960296,4.0,-0.900969,-0.433884,-0.129673,0.991557,-0.887331,0.461134,-0.617467,0.786597,0.022874,0.025247,0.024853,-0.002811,0.025685,26.496222,25.993972,25.085847,26.597589,25.737878,0.181807,0.185627,0.214820,0.180452,0.202823,0.010885,26.904028,26.737639,0.017089,26.135694,0.039859,25.505278,0.064275,25.075833,0.081256,25.685833,0.057221,25.551389,0.062469,25.129444,0.079121,26.706364,2.739894,0.179608,23.728528,31.570389,-0.022152,0.027071,-0.640273,0.781831,0.623490,-5.000000e-01,-0.866025,62.566986,0.214706,-0.145640,0.360346,22.855011,25.085847,27.316684,0.943945,-0.265828,0.024932,61.462530,0.195035,0.099346,0.011042,-0.006312,0.017353,0.022227,0.026922,0.024538,34.464496,35.535114,6.229096,47.014426,0.190569,0.025252,0.002403,0.950735,0.186978,0

##### Записываем сформированный датасет

In [205]:
# Записываем сформированный датасет
df_daily.to_csv('../data_archiv/DTG/DTG_new_fea_to_14_11_2025_1d_w128_noweekend.csv', index=False)

In [95]:
# Проверка на утечку данных в будущее. 
# Сравниваются датафреймы, созданные в разные даты.
df_1 = df_daily.copy()

In [96]:
df_2 = pd.read_csv('../data_archiv/DTG/DTG_new_fea_to_29_10_2025_1d_w92.csv')

In [97]:
cols = [c for c in df_1.columns if c not in ('date',)]
df_1 = df_1[:len(df_2)]

In [98]:
for col in cols:
    diff = round((df_1[col] - df_2[col]).abs().max(), 6)
    print("max abs diff on tail:", col, diff)

max abs diff on tail: open 0.0
max abs diff on tail: high 0.0
max abs diff on tail: low 0.0
max abs diff on tail: close 0.0
max abs diff on tail: DayAvgPrice 0.0
max abs diff on tail: IntradayStd 0.0
max abs diff on tail: Volume 0.0
max abs diff on tail: day_of_week 0
max abs diff on tail: day_of_year 0
max abs diff on tail: Log_Profit 0.0
max abs diff on tail: DayAvgPrice_diff 0.0
max abs diff on tail: DayAvgPrice_2diff 0.0
max abs diff on tail: POLY_1 0.0
max abs diff on tail: POLY_2 0.0
max abs diff on tail: POLY_3 0.0
max abs diff on tail: real_time 0.0
max abs diff on tail: imag_time 0.0
max abs diff on tail: c_week_real 0.0
max abs diff on tail: c_week_imag 0.0
max abs diff on tail: c_month_real 0.0
max abs diff on tail: c_month_imag 0.0
max abs diff on tail: c_quarter_real 0.0
max abs diff on tail: c_quarter_imag 0.0
max abs diff on tail: c_year_real 0.0
max abs diff on tail: c_year_imag 0.0
max abs diff on tail: parkinson_vol 0.0
max abs diff on tail: parkinson_vol_ma5 0.0
max 